# From unstructered data to it Provenance


<div style="text-align: right"> 
<h3>EScience - PPG IC/UFF 2017.1</h3>

<h4>Daniel Junior, Kid Valeriano</h4>

</div>


## Relembrando...
O projeto tem como base o artigo <strong>Provenance as data mining: Combining file system metadata with content analysis</strong>.
</hr>
<li>A ideia é utilizar técnicas de Mineração de Dados para reconstruir a proveniência de dados não-estruturados.</li>
<li>Os dados não-estruturados utilizados como dados de entrada para o projeto são arquivos textos contendo versões de um artigo até que o mesmo chegasse a sua edição final.</li>

## Arquitetura
<div width="100%" height="100%">
    <img src="images/arquitetura.png" height="400" width="800" style="margin: 0 auto;"/>
</div>

## Projeto
<ul>
    <li>Linguagem Python</li>
    <li>Biblioteca Sci-kit Learn</li>
    <li>Os dados usados no experimento são do repositório <strong><a href="https://l.facebook.com/l.php?u=http%3A%2F%2F137.207.234.78%2Fsearch%2Fdetail%2F&h=ATMRd4R-lyI7h1CWdhxvn6UWFuIQ6Sk5lh5mt_Yl5P4cZtgMLg2Eab3JBX_fKeG0-K69PJBwnMqh8BrtHvD__C71t0gmJg-NTkGuLNtxIehOx2CwfwQx_44sEEALKLVSY6y_fg">CiteSeerX</a></strong></li>
</ul>

## Projeto

Os passos a serem seguidos são:

<ol>
    <li>Dado um diretório que contenha as revisões de arquivos, devo ser capaz de recuperar o conteúdo de cada arquivo.</li>
    <li>Representar o conteúdo do arquivo de uma forma que seja possível aplicar algoritmos de Clustering.</li>
    <li>Executar o algoritmo para criar os clusters.</li>
    <li>Para cada arquivo em um cluster, recuperar os metados relativos ao dados de modificação do arquivo.</li>
    <li>Com os metadados em mãos, exportar para o PROV-Model, fazendo o armazenamento em um banco relacional (SQLite).</li>
    <li>Ser capaz de receber um arquivo de entrada e retornar uma representação gŕafica da proveniência do mesmo.</li>
</ol>


# Preparação de dados
Os dados obtiveram-se de um repositório chamado CiteSeerX, a qual tem os dados separados em dois tipos de formatos, todo o documento contido em um arquivo TXT e alguns características como o titulo, resumo, data de publicação, autores e as referências em um arquivo estruturado XML.

<div width="100%" height="100%">
    <img src="images/txt.png" height="50" width="500" style="margin: 0 auto;"/>
    <img src="images/xml.png" height="300" width="700" style="margin: 0 auto;"/>
</div>

### Passo 1: Carregar os dados

In [2]:
#imports
import os, time
from stat import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

CLUSTERS_NUMBER = 2

In [3]:
PATH = "data"
filenames = []
docs = []
for path, dirs, files in os.walk(PATH):
    for filename in files:
        fullpath = os.path.join(path, filename)
        with open(fullpath, 'r') as f:
            data = f.read()
            filenames.append(fullpath)
            docs.append(data)        

print(filenames[:10])

[]


Para realizar o trabalho precisa-se ter vários dados separados em listas com a finalidade de uma fácil análise: título do artigo, conteúdo do texto, referências do artigo, e o tempo nas que se fizeram as publicações.
Usando os 2 primeiro atributos para  realizar o agrupamento de documentos similares usando o algoritmo K-Means, e os últimos usados para reconstruir a procedência dos artigos analisados.

In [ ]:
titles = open('title_list.txt').read().split('\n')
titles = titles[:100]

synopses = open('textBody_list.txt').read().split('\n')
synopses = synopses[:100]

links = open('reference_list.txt').read().split('\n')
links = links[:100]

genres = open('time_list.txt').read().split('\n')
genres = genres[:100]

## Separação de dados

In [4]:
train_data = shuffle(docs, random_state=42)[int(len(docs)*0.2):]
test_data = shuffle(docs, random_state=42)[:int(len(docs)*0.2)]

train_filenames = shuffle(filenames, random_state=42)[int(len(docs)*0.2):]
test_filenames = shuffle(filenames, random_state=42)[:int(len(docs)*0.2)]


## Limpeza de dados

In [14]:
#nltk.download()
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [15]:
print (stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']


In [16]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english") #sacar la raiz de una palabra

## O que o torna mais eficiente

In [18]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english") #sacar la raiz de una palabra

### Separando palavras

In [ ]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

### Passo 2: Representação vetorial dos dados

In [ ]:
tfidf_vectorizer = ara

tfidf_matrix = tfidf_vectorizer.fit_transform(train_data)
print("Dimensões da matriz:")
print(tfidf_matrix.todense().shape)

### Passo 2: Representação vetorial dos dados

<div width="100%" height="100%">
    <img src="images/tfidf.png" style="margin: 0 auto;"/>
</div>

### Passo 3: Executar algoritmo de clustering

In [ ]:
km = KMeans(n_clusters=CLUSTERS_NUMBER, init='k-means++', max_iter=100, n_init=1,verbose=False)
km.fit(tfidf_matrix.todense())
print(km.labels_[:30])

### Passo 4: Recuperar os metadados dos arquivos de cada cluster

In [ ]:
clusters_predict = km.predict(tfidf_vectorizer.transform(test_data))
cluster_0_idx =  [i for i, x in enumerate(clusters_predict) if x == 0]
filenames_cluster_0 = [test_filenames[x] for x in cluster_0_idx ]

for filename in filenames_cluster_0[:3]:
    st = os.stat(filename)
    print("filename:", filename)
    print("file modified:", time.asctime(time.localtime(st[ST_CTIME])))
    print("file size:", st[ST_SIZE])
    

In [88]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(2):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()
    
#km.predict(tfidf_vectorizer.transform(test_data.data).todense())

Cluster 0: eduard coma yee baseballers organize teams games cs1 sublime lineups
Cluster 1: israeline israelis jfk eduard turklerin pera armenianism armeniaxn arabia jamea


In [89]:
from IPython.core.display import HTML
from urllib.request import urlopen
HTML(urlopen('https://raw.githubusercontent.com/lmarti/jupyter_custom/master/custom.include').read().decode('utf-8'))